In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import altair as alt
import os

import numpy as np
import warnings
warnings.simplefilter("ignore")

df = pd.read_csv('Data_mesh_cleaned.csv')


c:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df

,Filename,Content,Publisher
0,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,chieve effective aggregation localization jaga...,IEEE
1,Decentralized_Data_Governance_as_Part_of_a_Dat...,pproaches arif wider sumedha verma atif akhtar...,IEEE
2,Enterprise_Data_Strategy_A_Decentralized_Data_...,decentralized vijay kumar butte sujata butte i...,IEEE
3,Finding_Your_Way_Through_the_Jungle_of_Big_Dat...,rchitectures torsten priebe sebastian neumaier...,IEEE
4,978-1-4842-9253-2_1,look back architecture developed response pain...,Springer
5,978-1-4842-9253-2_10,provide high overview fabric evolution elabora...,Springer
6,978-1-4842-9253-2_11,saw fabric evolution previous architecture inf...,Springer
7,978-1-4842-9253-2_12,look intersection two initiative digital trans...,Springer
8,978-1-4842-9253-2_13,metadata digital era enterprise know aspect mu...,Springer
9,978-1-4842-9253-2_14,applying ai metadata intelligent cataloging qu...,Springer


In [3]:
def extract_topics(text, tfidf_vectorizer, lda_model):
    # TF-IDF Vectorization for the specific document
    tfidf_data = tfidf_vectorizer.transform([text])
    tfidf_values = tfidf_data.toarray()
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    df_tfidf = pd.DataFrame(data=tfidf_values, columns=feature_names)

    # Apply LDA
    doc_topic_prob = lda_model.transform(tfidf_data)

    # Get the most probable topic for the document
    most_probable_topic = np.argmax(doc_topic_prob)

    # Get the top words for the most probable topic with TF-IDF values
    topic = lda_model.components_[most_probable_topic]
    top_keywords_idx = topic.argsort()[:-10 - 1:-1]
    top_keywords = [feature_names[i] for i in top_keywords_idx]
    top_tfidf_values = [tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]
    topic_words = list(zip(top_keywords, top_tfidf_values))

    return most_probable_topic + 1, topic_words


In [4]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
tfidf_data = tfidf_vectorizer.fit_transform(df['Content'])

# Apply LDA
lda = LatentDirichletAllocation(n_components=20, random_state=42)
lda.fit(tfidf_data)

# Assuming 'df' is your DataFrame with 'pdf_content' column
df['topic'], df['topic_words'] = zip(*df.apply(lambda row: extract_topics(row['Content'], tfidf_vectorizer, lda), axis=1))

In [5]:
df

,Filename,Content,Publisher,topic,topic_words
0,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,chieve effective aggregation localization jaga...,IEEE,12,"[(localization, 3.995732273553991), (smart mon..."
1,Decentralized_Data_Governance_as_Part_of_a_Dat...,pproaches arif wider sumedha verma atif akhtar...,IEEE,8,"[(ai, 1.192371892647456), (fabric, 1.254892249..."
2,Enterprise_Data_Strategy_A_Decentralized_Data_...,decentralized vijay kumar butte sujata butte i...,IEEE,12,"[(localization, 3.995732273553991), (smart mon..."
3,Finding_Your_Way_Through_the_Jungle_of_Big_Dat...,rchitectures torsten priebe sebastian neumaier...,IEEE,3,"[(dairy, 3.995732273553991), (farm, 3.99573227..."
4,978-1-4842-9253-2_1,look back architecture developed response pain...,Springer,4,"[(hybrid cloud, 2.7429693050586232), (hybrid, ..."
5,978-1-4842-9253-2_10,provide high overview fabric evolution elabora...,Springer,13,"[(ml, 1.5978370007556204), (ai ml, 2.123930096..."
6,978-1-4842-9253-2_11,saw fabric evolution previous architecture inf...,Springer,10,"[(azure, 2.8971199848858813), (azure lake, 3.3..."
7,978-1-4842-9253-2_12,look intersection two initiative digital trans...,Springer,4,"[(hybrid cloud, 2.7429693050586232), (hybrid, ..."
8,978-1-4842-9253-2_13,metadata digital era enterprise know aspect mu...,Springer,19,"[(intelligent cataloging, 2.7429693050586232),..."
9,978-1-4842-9253-2_14,applying ai metadata intelligent cataloging qu...,Springer,9,"[(automated fabric aspect, 3.995732273553991),..."


In [6]:
result_df = pd.DataFrame(columns=['Filename', 'topic_word', 'idf_value'])

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    Filename = row['Filename']
    topic_words = row['topic_words']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_df = result_df.append({'Filename': Filename, 'topic_word': word, 'idf_value': idf_value}, ignore_index=True)

# Print the result DataFrame


In [7]:
result_df

,Filename,topic_word,idf_value
0,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,localization,3.995732
1,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,smart monitoring,3.995732
2,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,smart,2.203973
3,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,monitoring,2.123930
4,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,community,2.290984
...,...,...,...
385,urn_nbn_fi_uef-20211359,semantic knowledge graph,2.742969
386,urn_nbn_fi_uef-20211359,microservice,2.742969
387,urn_nbn_fi_uef-20211359,question,1.553385
388,urn_nbn_fi_uef-20211359,cdm,3.995732


In [8]:


# Select the top terms for each PDF
top_terms_df = result_df.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df['idf_value'] = top_terms_df['idf_value'] + np.random.rand(top_terms_df.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 2.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda = (heatmap + text).properties(width=1500)


In [9]:
chart_lda

alt.LayerChart(...)

In [10]:
result_df_publisher = pd.DataFrame(columns=['Publisher', 'topic_word', 'idf_value'])

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    Publisher = row['Publisher']
    topic_words = row['topic_words']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_df_publisher = result_df_publisher.append({'Publisher': Publisher, 'topic_word': word, 'idf_value': idf_value}, ignore_index=True)

# Print the result DataFrame

In [11]:
result_df_publisher

,Publisher,topic_word,idf_value
0,IEEE,localization,3.995732
1,IEEE,smart monitoring,3.995732
2,IEEE,smart,2.203973
3,IEEE,monitoring,2.123930
4,IEEE,community,2.290984
...,...,...,...
385,miscellaneous,semantic knowledge graph,2.742969
386,miscellaneous,microservice,2.742969
387,miscellaneous,question,1.553385
388,miscellaneous,cdm,3.995732


In [19]:


# Select the top terms for each PDF
top_terms_df = result_df_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df['idf_value'] = top_terms_df['idf_value'] + np.random.rand(top_terms_df.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 3.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publisher = (heatmap + text).properties(width=1500)
chart_lda_publisher

alt.LayerChart(...)

USING ONLY TFIDF

In [13]:
def find_top_words_for_each_document(df, num_top_words=10):

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))

    # Fit and transform the PDF content
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

    # Get feature names (words) from the TF-IDF vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame with TF-IDF values
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Now, for each document, find the top words based on TF-IDF scores
    top_words_for_each_document = []
    for i, row in enumerate(tfidf_df.iterrows()):
        _, document_tfidf_scores = row
        top_words_index = document_tfidf_scores.argsort()[-num_top_words:][::-1]
        top_words = [(feature_names[index], document_tfidf_scores[index]) for index in top_words_index]
        top_words_for_each_document.append(top_words)

    # Create a new DataFrame with 'pdf_content' and 'top_words'
    result_df = pd.DataFrame({'Filename': df['Filename'], 'Content': df['Content'], 'top_words': top_words_for_each_document})

    return result_df

In [14]:
# Call the function
result_df_2 = find_top_words_for_each_document(df)

# Display the result DataFrame
result_df_2

,Filename,Content,top_words
0,Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...,chieve effective aggregation localization jaga...,"[(localization, 0.6892272158019395), (region, ..."
1,Decentralized_Data_Governance_as_Part_of_a_Dat...,pproaches arif wider sumedha verma atif akhtar...,"[(port, 0.3186126815970601), (output port, 0.2..."
2,Enterprise_Data_Strategy_A_Decentralized_Data_...,decentralized vijay kumar butte sujata butte i...,"[(zone, 0.1432712960764015), (team, 0.11018636..."
3,Finding_Your_Way_Through_the_Jungle_of_Big_Dat...,rchitectures torsten priebe sebastian neumaier...,"[(logical warehouse, 0.20618519426800738), (da..."
4,978-1-4842-9253-2_1,look back architecture developed response pain...,"[(edw, 0.2474702114639632), (evolution, 0.1324..."
5,978-1-4842-9253-2_10,provide high overview fabric evolution elabora...,"[(fabric, 0.3201672087628716), (ai, 0.18458053..."
6,978-1-4842-9253-2_11,saw fabric evolution previous architecture inf...,"[(application, 0.20930639509369658), (fabric, ..."
7,978-1-4842-9253-2_12,look intersection two initiative digital trans...,"[(cloud, 0.38299706071513595), (hybrid cloud, ..."
8,978-1-4842-9253-2_13,metadata digital era enterprise know aspect mu...,"[(metadata, 0.342796111638865), (intelligent c..."
9,978-1-4842-9253-2_14,applying ai metadata intelligent cataloging qu...,"[(metadata, 0.21165264778046045), (automated, ..."


In [15]:
result_df_3 = pd.DataFrame(columns=['Filename', 'topic_word', 'tfidf_value'])

# Iterate through each row in the original DataFrame
for _, row in result_df_2.iterrows():
    Filename = row['Filename']
    top_words = row['top_words']

    # Iterate through each tuple in the list of top_words
    for word, tfidf in top_words:
        result_df_3 = result_df_3.append({'Filename': Filename, 'topic_word': word, 'tfidf_value': tfidf}, ignore_index=True)

# Display the result DataFrame
print(result_df_3)

                                              Filename     topic_word  \
0    Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...   localization   
1    Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...         region   
2    Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...          india   
3    Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...     regulation   
4    Breaking_Down_Data_Silos_Data_Mesh_to_Achieve_...         indian   
..                                                 ...            ...   
385                            urn_nbn_fi_uef-20211359      different   
386                            urn_nbn_fi_uef-20211359      interview   
387                            urn_nbn_fi_uef-20211359          study   
388                            urn_nbn_fi_uef-20211359          evans   
389                            urn_nbn_fi_uef-20211359  driven design   

     tfidf_value  
0       0.689227  
1       0.154530  
2       0.134843  
3       0.095560  
4       0.082798  
..       

In [16]:


# Select the top terms for each PDF
top_terms_df = result_df_3.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df['tfidf_value'] = top_terms_df['tfidf_value'] + np.random.rand(top_terms_df.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('tfidf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tfidf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.tfidf_value >= 0.2, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf = (heatmap + text).properties(width=1500)


In [17]:
chart_tfidf

alt.LayerChart(...)